In [99]:
# Na bazie OpenF1Api via https://openf1.org

In [100]:
# wyznaczenie liczby sezonów - lata od tzw "ery hybrydowej" 2014-2024
def seasons_count(end_year=2024):
    start_year = 2014
    seasons = list()
    for year in range(start_year, end_year + 1):
        seasons.append(year)
    return seasons

seasons = seasons_count(2024)
print(seasons)

[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]


In [101]:
# formatowanie pól daty i czasu

from datetime import datetime

def format_datetime_fields(datetime_string):
    dt = datetime.fromisoformat(datetime_string.replace('Z', '+00:00'))
    return dt.date().isoformat(), dt.time().isoformat()

In [102]:
# pobiernie meetiings - race week 
import requests
import pandas as pd

def get_all_meetings(start_season=2014, end_season=2024):
    meetings = []
    
    for year in range(start_season, end_season + 1):
        url = f"https://api.openf1.org/v1/meetings?year={year}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                for meeting in data:
                    date_start, time_start = format_datetime_fields(meeting['date_start'])
                    meetings.append({
                        "circuit_key": meeting['circuit_key'],
                        "circuit_short_name": meeting['circuit_short_name'],
                        "country_code": meeting['country_code'],
                        "country_key": meeting['country_key'],
                        "country_name": meeting['country_name'],
                        "date_start": date_start,
                        "time_start": time_start,
                        "gmt_offset": meeting['gmt_offset'],
                        "location": meeting['location'],
                        "meeting_key": meeting['meeting_key'],
                        "meeting_name": meeting['meeting_name'],
                        "meeting_official_name": meeting['meeting_official_name'],
                        "year": meeting['year'],
                    })
                print(f"Fetched {len(data)} meetings for {year}")
            else:
                print(f"No meetings found for {year}")
        else:
            print(f"Error fetching data for {year}: {response.status_code}")
    
    return pd.DataFrame(meetings)

In [103]:
all_meetings_df = get_all_meetings(2014, 2024)
all_meetings_df.head()

No meetings found for 2014
No meetings found for 2015
No meetings found for 2016
Error fetching data for 2017: 429
Error fetching data for 2018: 429
No meetings found for 2019
No meetings found for 2020
No meetings found for 2021
No meetings found for 2022
Fetched 23 meetings for 2023
Error fetching data for 2024: 429


,circuit_key,circuit_short_name,country_code,country_key,country_name,date_start,time_start,gmt_offset,location,meeting_key,meeting_name,meeting_official_name,year
0,63,Sakhir,BRN,36,Bahrain,2023-02-23,07:00:00,03:00:00,Sakhir,1140,Pre-Season Testing,FORMULA 1 ARAMCO PRE-SEASON TESTING 2023,2023
1,63,Sakhir,BRN,36,Bahrain,2023-03-03,11:30:00,03:00:00,Sakhir,1141,Bahrain Grand Prix,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2023,2023
2,149,Jeddah,KSA,153,Saudi Arabia,2023-03-17,13:30:00,03:00:00,Jeddah,1142,Saudi Arabian Grand Prix,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2023,2023
3,10,Melbourne,AUS,5,Australia,2023-03-31,01:30:00,11:00:00,Melbourne,1143,Australian Grand Prix,FORMULA 1 ROLEX AUSTRALIAN GRAND PRIX 2023,2023
4,144,Baku,AZE,30,Azerbaijan,2023-04-28,09:30:00,04:00:00,Baku,1207,Azerbaijan Grand Prix,FORMULA 1 AZERBAIJAN GRAND PRIX 2023,2023


In [104]:
# pobierz kolumny z tabeli meetings
meetings_columns = all_meetings_df.columns.tolist()
print("Meetings columns:", meetings_columns)

Meetings columns: ['circuit_key', 'circuit_short_name', 'country_code', 'country_key', 'country_name', 'date_start', 'time_start', 'gmt_offset', 'location', 'meeting_key', 'meeting_name', 'meeting_official_name', 'year']


In [105]:
meetings_key_unique = all_meetings_df['meeting_key'].unique()
print(meetings_key_unique)

[1140 1141 1142 1143 1207 1208 1210 1211 1212 1213 1214 1215 1216 1217
 1218 1219 1220 1221 1222 1223 1224 1225 1226]


In [106]:
# pobranie poszczególnych wyścigów w określonych sezonach - sessions
import requests
import pandas as pd
from time import sleep

def get_all_sessions(meetings):
    sessions = []
    
    for meeting in meetings_key_unique:
        url = f"https://api.openf1.org/v1/sessions?meeting_key={meeting}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                for session in data:
                    # Parse dates and times
                    date_start, time_start = format_datetime_fields(session['date_start'])
                    date_end, time_end = format_datetime_fields(session['date_end'])
                    
                    sessions.append({
                        'meeting_key': session['meeting_key'],
                        'year': session['year'],
                        'session_key': session['session_key'],
                        'session_type': session['session_type'],
                        'session_name': session['session_name'],
                        'date_start': date_start,
                        'time_start': time_start,
                        'date_end': date_end,
                        'time_end': time_end,
                        'gmt_offset': session['gmt_offset'],
                        'location': session['location'],
                        'country_name': session['country_name'],
                        'country_code': session['country_code'],
                        'circuit_short_name': session['circuit_short_name'],
                    })
                
                print(f"Fetched {len(data)} sessions for {meeting}")
            else:
                print(f"No sessions found for {meeting}")
        else:
            print(f"Error fetching data for {meeting}: {response.status_code}")
        sleep(1)
    return pd.DataFrame(sessions)

# Pobierz wszystkie sesje 2014–2024
all_sessions_df = get_all_sessions(meetings_key_unique)

Fetched 3 sessions for 1140
Fetched 5 sessions for 1141
Fetched 5 sessions for 1142
Fetched 5 sessions for 1143
Fetched 5 sessions for 1207
Fetched 5 sessions for 1208
Fetched 5 sessions for 1210
Fetched 5 sessions for 1211
Fetched 5 sessions for 1212
Fetched 5 sessions for 1213
Fetched 5 sessions for 1214
Fetched 5 sessions for 1215
Fetched 5 sessions for 1216
Fetched 5 sessions for 1217
Fetched 5 sessions for 1218
Fetched 5 sessions for 1219
Fetched 5 sessions for 1220
Fetched 5 sessions for 1221
Fetched 5 sessions for 1222
Fetched 5 sessions for 1223
Fetched 5 sessions for 1224
Fetched 5 sessions for 1225
Fetched 5 sessions for 1226


In [107]:
all_sessions_df.head()

,meeting_key,year,session_key,session_type,session_name,date_start,time_start,date_end,time_end,gmt_offset,location,country_name,country_code,circuit_short_name
0,1140,2023,9222,Practice,Practice 1,2023-02-23,07:00:00,2023-02-23,16:30:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir
1,1140,2023,7763,Practice,Practice 2,2023-02-24,07:00:00,2023-02-24,16:30:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir
2,1140,2023,7764,Practice,Practice 3,2023-02-25,07:00:00,2023-02-25,16:30:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir
3,1141,2023,7765,Practice,Practice 1,2023-03-03,11:30:00,2023-03-03,12:30:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir
4,1141,2023,7766,Practice,Practice 2,2023-03-03,15:00:00,2023-03-03,16:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir


In [108]:
all_sessions_df.tail(100)

,meeting_key,year,session_key,session_type,session_name,date_start,time_start,date_end,time_end,gmt_offset,location,country_name,country_code,circuit_short_name
13,1143,2023,7780,Practice,Practice 1,2023-03-31,01:30:00,2023-03-31,02:30:00,11:00:00,Melbourne,Australia,AUS,Melbourne
14,1143,2023,7781,Practice,Practice 2,2023-03-31,05:00:00,2023-03-31,06:00:00,11:00:00,Melbourne,Australia,AUS,Melbourne
15,1143,2023,7782,Practice,Practice 3,2023-04-01,01:30:00,2023-04-01,02:30:00,11:00:00,Melbourne,Australia,AUS,Melbourne
16,1143,2023,7783,Qualifying,Qualifying,2023-04-01,05:00:00,2023-04-01,06:00:00,11:00:00,Melbourne,Australia,AUS,Melbourne
17,1143,2023,7787,Race,Race,2023-04-02,05:00:00,2023-04-02,07:00:00,10:00:00,Melbourne,Australia,AUS,Melbourne
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,1226,2023,9190,Practice,Practice 1,2023-11-24,09:30:00,2023-11-24,10:30:00,04:00:00,Yas Island,United Arab Emirates,UAE,Yas Marina Circuit
109,1226,2023,9191,Practice,Practice 2,2023-11-24,13:00:00,2023-11-24,14:00:00,04:00:00,Yas Island,United Arab Emirates,UAE,Yas Marina Circuit
110,1226,2023,9192,Practice,Practice 3,2023-11-25,10:30:00,2023-11-25,11:30:00,04:00:00,Yas Island,United Arab Emirates,UAE,Yas Marina Circuit
111,1226,2023,9193,Qualifying,Qualifying,2023-11-25,14:00:00,2023-11-25,15:00:00,04:00:00,Yas Island,United Arab Emirates,UAE,Yas Marina Circuit


In [109]:
# pobranie kolumn z tabeli sessions
sessions_columns = all_sessions_df.columns.tolist()
print("Sessions columns:", sessions_columns)

Sessions columns: ['meeting_key', 'year', 'session_key', 'session_type', 'session_name', 'date_start', 'time_start', 'date_end', 'time_end', 'gmt_offset', 'location', 'country_name', 'country_code', 'circuit_short_name']


In [110]:
sessions_key_unique= all_sessions_df['session_key'].unique()
print(sessions_key_unique)

[9222 7763 7764 7765 7766 7767 7768 7953 7772 7773 7774 7775 7779 7780
 7781 7782 7783 7787 9063 9064 9278 9069 9070 9071 9072 9073 9074 9078
 9087 9088 9089 9090 9094 9095 9096 9097 9098 9102 9103 9104 9105 9106
 9110 9111 9112 9282 9117 9118 9119 9120 9121 9122 9126 9223 9127 9128
 9129 9133 9134 9135 9286 9140 9141 9142 9143 9144 9145 9149 9150 9151
 9152 9153 9157 9158 9159 9160 9161 9165 9166 9167 9168 9169 9173 9214
 9215 9298 9220 9221 9206 9207 9294 9212 9213 9174 9175 9176 9177 9181
 9198 9304 9308 9204 9205 9182 9183 9184 9314 9189 9190 9191 9192 9193
 9197]


In [111]:
# pobranie kierowcow poszczegolnych zespołów dka kazdego z meeting w sessions

def get_all_drivers(unique_meetings_keys):
    drivers = []
    for meeting_key in unique_meetings_keys:
        url = f"https://api.openf1.org/v1/drivers?meeting_key={meeting_key}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                for driver in data:
                    drivers.append({
                        'driver_number': driver.get('driver_number'),
                        'broadcast_name': driver.get('broadcast_name'),
                        'full_name': driver.get('full_name'),
                        'first_name': driver.get('first_name'),
                        'last_name': driver.get('last_name'),
                        'name_acronym': driver.get('name_acronym'),
                        'country_code': driver.get('country_code'),
                        'team_name': driver.get('team_name'),
                        'meeting_key': meeting_key,
                        'session_key': driver.get('session_key')
                    })
                print(f"Fetched {len(data)} drivers for meeting {meeting_key}")
            else:
                print(f"No drivers found for meeting {meeting_key}")
        else:
            print(f"Error fetching data for meeting {meeting_key}: {response.status_code}")
        sleep(1)
    return pd.DataFrame(drivers)

In [112]:
drivers_df = get_all_drivers(meetings_key_unique)

Fetched 53 drivers for meeting 1140
Fetched 100 drivers for meeting 1141
Fetched 100 drivers for meeting 1142
Fetched 100 drivers for meeting 1143
Fetched 99 drivers for meeting 1207
Fetched 100 drivers for meeting 1208
Fetched 100 drivers for meeting 1210
Fetched 100 drivers for meeting 1211
Fetched 100 drivers for meeting 1212
Fetched 100 drivers for meeting 1213
Fetched 100 drivers for meeting 1214
Fetched 115 drivers for meeting 1215
Fetched 100 drivers for meeting 1216
Fetched 100 drivers for meeting 1217
Fetched 100 drivers for meeting 1218
Fetched 99 drivers for meeting 1219
Fetched 100 drivers for meeting 1220
Fetched 100 drivers for meeting 1221
Fetched 100 drivers for meeting 1222
Fetched 100 drivers for meeting 1223
Fetched 100 drivers for meeting 1224
Fetched 100 drivers for meeting 1225
Fetched 100 drivers for meeting 1226


In [113]:
drivers_df.head()

,driver_number,broadcast_name,full_name,first_name,last_name,name_acronym,country_code,team_name,meeting_key,session_key
0,1,M VERSTAPPEN,Max VERSTAPPEN,Max,Verstappen,VER,NED,Red Bull Racing,1140,7763
1,2,L SARGEANT,Logan SARGEANT,Logan,Sargeant,SAR,USA,Williams,1140,7763
2,4,L NORRIS,Lando NORRIS,Lando,Norris,NOR,GBR,McLaren,1140,7763
3,10,P GASLY,Pierre GASLY,Pierre,Gasly,GAS,FRA,Alpine,1140,7763
4,11,S PEREZ,Sergio PEREZ,Sergio,Perez,PER,MEX,Red Bull Racing,1140,7763


In [114]:
# pobranie kolumn z tabeli drivers
drivers_columns = drivers_df.columns.tolist()
print("Drivers columns:", drivers_columns)

Drivers columns: ['driver_number', 'broadcast_name', 'full_name', 'first_name', 'last_name', 'name_acronym', 'country_code', 'team_name', 'meeting_key', 'session_key']


In [115]:
unique_drivers = drivers_df['broadcast_name'].unique()

In [116]:
print(unique_drivers)

['M VERSTAPPEN' 'L SARGEANT' 'L NORRIS' 'P GASLY' 'S PEREZ' 'F ALONSO'
 'C LECLERC' 'K MAGNUSSEN' 'N DE VRIES' 'Y TSUNODA' 'G ZHOU'
 'N HULKENBERG' 'E OCON' 'L HAMILTON' 'C SAINZ' 'G RUSSELL' 'O PIASTRI'
 'A ALBON' 'F DRUGOVICH' 'V BOTTAS' 'L STROLL' 'D RICCIARDO' 'P ARON'
 'D BEGANOVIC' "Z O'SULLIVAN" 'L FORNAROLI' 'F COLAPINTO' 'M BOYA'
 'S MONTOYA' 'L BROWNING' 'R VILLAGOMEZ' 'O GRAY' 'I COHEN' 'J MARTI'
 'C MANSELL' 'T BARNARD' 'W SHIN' 'G BORTOLETO' 'O GOETHE' 'K FREDERICK'
 'G SAUCY' 'N TSOLOV' 'J EDGAR' 'G MINI' 'C COLLET' 'T SMITH' 'M ESTERSON'
 'H BARTER' 'N BEDRIN' 'A GARCIA' 'S FLOERSCH' 'R FARIA' 'R SHWARTZMAN'
 'L LAWSON' 'I HADJAR' 'F VESTI' 'O BEARMAN' 'J DOOHAN' 'T POURCHAIRE'
 "P O'WARD" 'J DENNIS']


In [117]:
# Funkcja do pobierania końcowej tabeli wyników dla sesji
def get_final_session_results(session_keys, include_driver_details=True):
    all_results = []
    
    for session_key in session_keys:
        url = f"https://api.openf1.org/v1/position?session_key={session_key}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                # Sortuj dane według daty (najnowsze na końcu)
                data_sorted = sorted(data, key=lambda x: x.get('date', ''))
                
                # Znajdź ostatnie pozycje dla każdego kierowcy
                latest_positions = {}
                for record in data_sorted:
                    driver_number = record.get('driver_number')
                    if driver_number:
                        latest_positions[driver_number] = record
                
                date, time = format_datetime_fields(data_sorted[-1]['date'])
                # Dodaj wszystkie końcowe pozycje do wyników
                for driver_number, record in latest_positions.items():
                    result = {
                        'session_key': session_key,
                        'driver_number': driver_number,
                        'position': record.get('position'),
                        'date': date,
                        'time': time,
                        'meeting_key': record.get('meeting_key')
                    }
                    all_results.append(result)
                
                print(f"Session {session_key}: Pobrano {len(latest_positions)} końcowych pozycji")
            else:
                print(f"Session {session_key}: Brak danych pozycji")
        else:
            print(f"Error fetching data for session {session_key}: {response.status_code}")
        
        sleep(0.5)  # Krótka pauza między requestami
    
    results_df = pd.DataFrame(all_results)
    
    if not results_df.empty and include_driver_details:
        # Połącz z danymi o sesjach
        results_df = results_df.merge(
            all_sessions_df[['session_key', 'session_name', 'year', 'location', 'country_name', 'circuit_short_name']], 
            on='session_key', 
            how='left'
        )
        
        # Połącz z danymi o kierowcach używając session_key zamiast meeting_key
        results_df = results_df.merge(
            drivers_df[['driver_number', 'session_key', 'broadcast_name', 'full_name', 'team_name']].drop_duplicates(), 
            on=['driver_number', 'session_key'], 
            how='left'
        )
    
    return results_df

In [ ]:
results_df = get_final_session_results(sessions_key_unique, include_driver_details=True)
results_df.head()

Session 9222: Pobrano 19 końcowych pozycji
Session 7763: Pobrano 17 końcowych pozycji
Session 7764: Pobrano 17 końcowych pozycji
Session 7765: Pobrano 20 końcowych pozycji
Session 7766: Pobrano 20 końcowych pozycji
Session 7767: Pobrano 20 końcowych pozycji
Session 7768: Pobrano 20 końcowych pozycji
Session 7953: Pobrano 20 końcowych pozycji
Session 7772: Pobrano 20 końcowych pozycji
Session 7773: Pobrano 20 końcowych pozycji
Session 7774: Pobrano 20 końcowych pozycji
Session 7775: Pobrano 20 końcowych pozycji
Session 7779: Pobrano 20 końcowych pozycji
Session 7780: Pobrano 20 końcowych pozycji
Session 7781: Pobrano 20 końcowych pozycji
Session 7782: Pobrano 20 końcowych pozycji
Session 7783: Pobrano 20 końcowych pozycji
Session 7787: Pobrano 20 końcowych pozycji
Session 9063: Pobrano 20 końcowych pozycji
Session 9064: Pobrano 20 końcowych pozycji
Session 9278: Pobrano 20 końcowych pozycji
Session 9069: Pobrano 19 końcowych pozycji
Session 9070: Pobrano 20 końcowych pozycji
Session 907

In [ ]:
# posortuj wyniki według sesji i pozycji
def sort_results_by_session_and_position(results_df):
    return results_df.sort_values(by=['session_key', 'position'])

sorted_results_df = sort_results_by_session_and_position(results_df)
sorted_results_df.head()

,session_key,driver_number,position,date,time,meeting_key,session_name,year,location,country_name,circuit_short_name,broadcast_name,full_name,team_name
2361,9461,16,1,2024-12-06,10:27:18.161000,1252,Practice 1,2024,Yas Island,United Arab Emirates,Yas Marina Circuit,C LECLERC,Charles LECLERC,Ferrari
2357,9461,4,2,2024-12-06,10:27:18.161000,1252,Practice 1,2024,Yas Island,United Arab Emirates,Yas Marina Circuit,L NORRIS,Lando NORRIS,McLaren
2372,9461,44,3,2024-12-06,10:27:18.161000,1252,Practice 1,2024,Yas Island,United Arab Emirates,Yas Marina Circuit,L HAMILTON,Lewis HAMILTON,Mercedes
2375,9461,63,4,2024-12-06,10:27:18.161000,1252,Practice 1,2024,Yas Island,United Arab Emirates,Yas Marina Circuit,G RUSSELL,George RUSSELL,Mercedes
2358,9461,10,5,2024-12-06,10:27:18.161000,1252,Practice 1,2024,Yas Island,United Arab Emirates,Yas Marina Circuit,P GASLY,Pierre GASLY,Alpine


In [ ]:
# Funkcje do analizy końcowych wyników sesji
def get_session_type_results(session_type, max_sessions=None):
    
    sessions = all_sessions_df[all_sessions_df['session_name'] == session_type]
    
    if max_sessions:
        sessions = sessions.head(max_sessions)
    
    session_keys = sessions['session_key'].tolist()
    
    print(f"Pobieranie wyników dla {len(session_keys)} sesji typu '{session_type}'")
    
    return get_final_session_results(session_keys)


In [ ]:
# KOMPLETNE POBIERANIE RANKINGU DLA WSZYSTKICH SESJI
# Funkcja do pobierania rankingu dla wszystkich dostępnych sesji każdego typu

def get_all_rankings_by_session_type():

    session_types = all_sessions_df['session_name'].unique()
    all_rankings = {}
    
    
    for session_type in session_types:
        print(f"Przetwarzanie: {session_type}")
        
        # Pobierz wszystkie sesje tego typu
        sessions_of_type = all_sessions_df[all_sessions_df['session_name'] == session_type]
        session_keys = sessions_of_type['session_key'].tolist()
        
        print(f"   Znaleziono {len(session_keys)} sesji typu '{session_type}'")
        
        # Pobierz wyniki dla wszystkich sesji tego typu
        rankings = get_final_session_results(session_keys, include_driver_details=True)
        
        if not rankings.empty:
            # Sortuj według sesji i pozycji
            rankings_sorted = rankings.sort_values(['session_key', 'position'])
            all_rankings[session_type] = rankings_sorted
            
            print(f"Pobrano {len(rankings)} wyników z {rankings['session_key'].nunique()} sesji")
            print(f"Lata: {rankings['year'].min()}-{rankings['year'].max()}")
            print(f"Pozycje: {rankings['position'].min()}-{rankings['position'].max()}")
        else:
            print(f"Brak wyników dla {session_type}")
        
        print()
    
    return all_rankings

In [ ]:
# Zwraca ramkę danych dla każdego meeting ze wszystkimi sesjami i końcowymi pozycjami kierowców

def get_meeting_all_sessions_positions():

    # Pobierz wszystkie session_keys
    all_session_keys = all_sessions_df['session_key'].tolist()
    
    # Pobierz końcowe pozycje ze wszystkich sesji
    all_results = get_final_session_results(all_session_keys, include_driver_details=True)
    
    # Połącz dane meeting z sesjami
    meeting_sessions = all_meetings_df.merge(
        all_sessions_df[['meeting_key', 'session_key', 'session_name', 'session_type']], 
        on='meeting_key', 
        how='inner'
    )
    
    # Połącz z wynikami wszystkich sesji
    final_data = meeting_sessions.merge(
        all_results[['session_key', 'driver_number', 'position', 'broadcast_name', 'full_name', 'team_name']], 
        on='session_key', 
        how='inner'  # inner join żeby mieć tylko te z pozycjami
    )
    
    return final_data

In [ ]:
meeting_all_sessions_positions = get_meeting_all_sessions_positions()
meeting_all_sessions_positions.head()

Session 9462: Pobrano 20 końcowych pozycji
Session 9463: Pobrano 20 końcowych pozycji
Session 9464: Pobrano 20 końcowych pozycji
Session 9465: Pobrano 20 końcowych pozycji
Session 9466: Pobrano 20 końcowych pozycji
Session 9467: Pobrano 20 końcowych pozycji
Session 9468: Pobrano 20 końcowych pozycji
Session 9472: Pobrano 20 końcowych pozycji
Session 9473: Pobrano 20 końcowych pozycji
Session 9474: Pobrano 20 końcowych pozycji
Session 9475: Pobrano 20 końcowych pozycji
Session 9476: Pobrano 20 końcowych pozycji
Session 9480: Pobrano 20 końcowych pozycji
Session 9481: Pobrano 20 końcowych pozycji
Session 9482: Pobrano 20 końcowych pozycji
Session 9483: Pobrano 19 końcowych pozycji
Session 9484: Pobrano 19 końcowych pozycji
Session 9488: Pobrano 19 końcowych pozycji
Session 9489: Pobrano 20 końcowych pozycji
Session 9490: Pobrano 20 końcowych pozycji
Session 9491: Pobrano 20 końcowych pozycji
Session 9492: Pobrano 20 końcowych pozycji
Session 9496: Pobrano 20 końcowych pozycji
Session 966

,circuit_key,circuit_short_name,country_code,country_key,country_name,date_start,time_start,gmt_offset,location,meeting_key,...,meeting_official_name,year,session_key,session_name,session_type,driver_number,position,broadcast_name,full_name,team_name
0,63,Sakhir,BRN,36,Bahrain,2024-02-21,07:00:00,03:00:00,Sakhir,1228,...,FORMULA 1 ARAMCO PRE-SEASON TESTING 2024,2024,9462,Practice 1,Practice,1,1,M VERSTAPPEN,Max VERSTAPPEN,Red Bull Racing
1,63,Sakhir,BRN,36,Bahrain,2024-02-21,07:00:00,03:00:00,Sakhir,1228,...,FORMULA 1 ARAMCO PRE-SEASON TESTING 2024,2024,9462,Practice 1,Practice,2,11,L SARGEANT,Logan SARGEANT,Williams
2,63,Sakhir,BRN,36,Bahrain,2024-02-21,07:00:00,03:00:00,Sakhir,1228,...,FORMULA 1 ARAMCO PRE-SEASON TESTING 2024,2024,9462,Practice 1,Practice,3,4,D RICCIARDO,Daniel RICCIARDO,RB
3,63,Sakhir,BRN,36,Bahrain,2024-02-21,07:00:00,03:00:00,Sakhir,1228,...,FORMULA 1 ARAMCO PRE-SEASON TESTING 2024,2024,9462,Practice 1,Practice,4,2,L NORRIS,Lando NORRIS,McLaren
4,63,Sakhir,BRN,36,Bahrain,2024-02-21,07:00:00,03:00:00,Sakhir,1228,...,FORMULA 1 ARAMCO PRE-SEASON TESTING 2024,2024,9462,Practice 1,Practice,10,5,P GASLY,Pierre GASLY,Alpine


In [ ]:
# pobierz początkową pozycję dla każdego z kierowców w meetings dla każdej sesji
def get_starting_session_position(session_keys, include_driver_details=True):
    all_results = []
    
    for session_key in session_keys:
        url = f"https://api.openf1.org/v1/position?session_key={session_key}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                # Sortuj dane według daty (najstarsze na początku)
                data_sorted = sorted(data, key=lambda x: x.get('date', ''))
                
                # Znajdź pierwsze pozycje dla każdego kierowcy
                first_positions = {}
                for record in data_sorted:
                    driver_number = record.get('driver_number')
                    if driver_number and driver_number not in first_positions:
                        first_positions[driver_number] = record
                
                date, time = format_datetime_fields(data_sorted[0]['date'])
                # Dodaj wszystkie początkowe pozycje do wyników
                for driver_number, record in first_positions.items():
                    result = {
                        'session_key': session_key,
                        'driver_number': driver_number,
                        'position': record.get('position'),
                        'date': date,
                        'time': time,
                        'meeting_key': record.get('meeting_key')
                    }
                    all_results.append(result)
                
                print(f"Session {session_key}: Pobrano {len(first_positions)} początkowych pozycji")
            else:
                print(f"Session {session_key}: Brak danych pozycji")
        else:
            print(f"Error fetching data for session {session_key}: {response.status_code}")
        
        sleep(0.5)  # Krótka pauza między requestami
    
    results_df = pd.DataFrame(all_results)
    
    if not results_df.empty and include_driver_details:
        # Połącz z danymi o sesjach
        results_df = results_df.merge(
            all_sessions_df[['session_key', 'session_name', 'year', 'location', 'country_name']], 
            on='session_key', 
            how='left'
        )
        
        # Połącz z danymi o kierowcach używając session_key zamiast meeting_key
        results_df = results_df.merge(
            drivers_df[['driver_number', 'session_key', 'broadcast_name', 'full_name', 'team_name']].drop_duplicates(), 
            on=['driver_number', 'session_key'], 
            how='left'
        )
    
    return results_df


In [ ]:
starting_position_df = get_starting_session_position(sessions_key_unique, include_driver_details=True)
starting_position_df.head()

Session 9462: Pobrano 20 początkowych pozycji
Session 9463: Pobrano 20 początkowych pozycji
Session 9464: Pobrano 20 początkowych pozycji
Session 9465: Pobrano 20 początkowych pozycji
Session 9466: Pobrano 20 początkowych pozycji
Session 9467: Pobrano 20 początkowych pozycji
Session 9468: Pobrano 20 początkowych pozycji
Session 9472: Pobrano 20 początkowych pozycji
Session 9473: Pobrano 20 początkowych pozycji
Session 9474: Pobrano 20 początkowych pozycji
Session 9475: Pobrano 20 początkowych pozycji
Session 9476: Pobrano 20 początkowych pozycji
Session 9480: Pobrano 20 początkowych pozycji
Session 9481: Pobrano 20 początkowych pozycji
Session 9482: Pobrano 20 początkowych pozycji
Session 9483: Pobrano 19 początkowych pozycji
Session 9484: Pobrano 19 początkowych pozycji
Session 9488: Pobrano 19 początkowych pozycji
Session 9489: Pobrano 20 początkowych pozycji
Session 9490: Pobrano 20 początkowych pozycji
Session 9491: Pobrano 20 początkowych pozycji
Session 9492: Pobrano 20 początkow

,session_key,driver_number,position,date,time,meeting_key,session_name,year,location,country_name,broadcast_name,full_name,team_name
0,9462,1,1,2024-02-21,06:55:17.159000,1228,Practice 1,2024,Sakhir,Bahrain,M VERSTAPPEN,Max VERSTAPPEN,Red Bull Racing
1,9462,2,2,2024-02-21,06:55:17.159000,1228,Practice 1,2024,Sakhir,Bahrain,L SARGEANT,Logan SARGEANT,Williams
2,9462,3,3,2024-02-21,06:55:17.159000,1228,Practice 1,2024,Sakhir,Bahrain,D RICCIARDO,Daniel RICCIARDO,RB
3,9462,4,4,2024-02-21,06:55:17.159000,1228,Practice 1,2024,Sakhir,Bahrain,L NORRIS,Lando NORRIS,McLaren
4,9462,10,5,2024-02-21,06:55:17.159000,1228,Practice 1,2024,Sakhir,Bahrain,P GASLY,Pierre GASLY,Alpine


In [ ]:
# dla kazdego meeting w session pobierz pit stop dla kazdego kierowcy gdzie pit duration jest rozny od null

def get_pit_stops_for_sessions(session_key):
    pit_stops = []
    
    for session_key in session_key:
        url = f"https://api.openf1.org/v1/pit?session_key={session_key}&pit_duration>0"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            pit_date, pit_time = format_datetime_fields(data[0]['date']) if data else (None, None)

            if isinstance(data, list) and len(data) > 0:
                for pit in data:
                    pit_stops.append({
                        'meeting_key': pit.get('meeting_key'),
                        'session_key': session_key,
                        'driver_number': pit.get('driver_number'),
                        'lap': pit.get('lap'),
                        'pit_duration': pit.get('pit_duration'),
                        'date': pit_date,
                        'time': pit_time,
                    })
                print(f"Fetched {len(data)} pit stops for session {session_key}")
            else:
                print(f"No pit stops found for session {session_key}")
        else:
            print(f"Error fetching data for session {session_key}: {response.status_code}")
        
        sleep(0.5)
    
    return pd.DataFrame(pit_stops)


In [ ]:
pit_stops_df = get_pit_stops_for_sessions(sessions_key_unique)
pit_stops_df.head()

Fetched 197 pit stops for session 9462
Fetched 191 pit stops for session 9463
Fetched 192 pit stops for session 9464
Fetched 58 pit stops for session 9465
Fetched 59 pit stops for session 9466
Fetched 64 pit stops for session 9467
Fetched 72 pit stops for session 9468
Fetched 43 pit stops for session 9472
Fetched 54 pit stops for session 9473
Fetched 57 pit stops for session 9474
Fetched 44 pit stops for session 9475
Fetched 73 pit stops for session 9476
Fetched 19 pit stops for session 9480
Fetched 62 pit stops for session 9481
Fetched 57 pit stops for session 9482
Fetched 67 pit stops for session 9483
Fetched 67 pit stops for session 9484
Fetched 36 pit stops for session 9488
Fetched 59 pit stops for session 9489
Fetched 17 pit stops for session 9490
Fetched 56 pit stops for session 9491
Fetched 55 pit stops for session 9492
Fetched 54 pit stops for session 9496
Fetched 35 pit stops for session 9663
Fetched 33 pit stops for session 9668
Fetched 1 pit stops for session 9672
Fetched 74

,meeting_key,session_key,driver_number,lap,pit_duration,date,time
0,1228,9462,31,None,30.4,2024-02-21,07:03:30.378000
1,1228,9462,77,None,471.2,2024-02-21,07:03:30.378000
2,1228,9462,81,None,29.3,2024-02-21,07:03:30.378000
3,1228,9462,63,None,34.6,2024-02-21,07:03:30.378000
4,1228,9462,23,None,424.2,2024-02-21,07:03:30.378000


In [ ]:
pit_stops_agg = (
    pit_stops_df
    .groupby(['session_key', 'driver_number'])
    .agg(
        pit_stop_count=('pit_duration', 'count'),
        avg_pit_duration=('pit_duration', 'mean')
    )
    .reset_index()
)

In [ ]:
pit_stops_agg.head()

,session_key,driver_number,pit_stop_count,avg_pit_duration
0,9461,4,3,376.133333
1,9461,10,4,264.900000
2,9461,11,3,302.466667
3,9461,14,4,302.300000
4,9461,16,1,22.200000


In [ ]:
# https://api.openf1.org/v1/race_control
# get all race control for all session in meetings

def get_race_control(session_keys):
    """
    Pobiera wszystkie dane race control dla określonych sesji
    """
    all_race_control = []
    
    for session_key in session_keys:
        url = f"https://api.openf1.org/v1/race_control?session_key={session_key}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                for control in data:
                    # Formatuj datę i czas
                    date, time = format_datetime_fields(control['date'])
                    
                    race_control_record = {
                        'session_key': session_key,
                        'date': date,
                        'time': time,
                        'category': control.get('category'),
                        'flag': control.get('flag'),
                        'lap_number': control.get('lap_number'),
                        'message': control.get('message')
                    }
                    all_race_control.append(race_control_record)
                
                print(f"Session {session_key}: Pobrano {len(data)} wiadomości race control")
            else:
                print(f"Session {session_key}: Brak danych race control")
        else:
            print(f"Error fetching race control for session {session_key}: {response.status_code}")
        
        sleep(0.5)
    
    return pd.DataFrame(all_race_control)

In [ ]:
race_controlls_df = get_race_control(sessions_key_unique)
race_controlls_df.head()

Session 9462: Pobrano 22 wiadomości race control
Session 9463: Pobrano 17 wiadomości race control
Session 9464: Pobrano 23 wiadomości race control
Session 9465: Pobrano 11 wiadomości race control
Session 9466: Pobrano 8 wiadomości race control
Session 9467: Pobrano 5 wiadomości race control
Session 9468: Pobrano 14 wiadomości race control
Session 9472: Pobrano 69 wiadomości race control
Session 9473: Pobrano 6 wiadomości race control
Session 9474: Pobrano 10 wiadomości race control
Session 9475: Pobrano 21 wiadomości race control
Session 9476: Pobrano 35 wiadomości race control
Session 9480: Pobrano 66 wiadomości race control
Session 9481: Pobrano 19 wiadomości race control
Session 9482: Pobrano 25 wiadomości race control
Session 9483: Pobrano 18 wiadomości race control
Session 9484: Pobrano 41 wiadomości race control
Session 9488: Pobrano 82 wiadomości race control
Session 9489: Pobrano 15 wiadomości race control
Session 9490: Pobrano 12 wiadomości race control
Session 9491: Pobrano 9

,session_key,date,time,category,flag,lap_number,message
0,9462,2024-02-21,07:00:00,Flag,GREEN,NaN,GREEN LIGHT - PIT EXIT OPEN
1,9462,2024-02-21,08:40:55,Other,None,NaN,PIT EXIT CLOSED
2,9462,2024-02-21,08:41:10,Flag,GREEN,NaN,GREEN LIGHT - PIT EXIT OPEN
3,9462,2024-02-21,10:30:29,Other,None,NaN,THIS IS A TEST MESSAGE
4,9462,2024-02-21,10:38:14,Flag,YELLOW,NaN,YELLOW IN TRACK SECTOR 3


In [ ]:
# pobierz rodzaje opon 
# https://api.openf1.org/v1/stints?session_key={session_key}
#     "compound": "SOFT",
#     "driver_number": 16,
#     "lap_end": 20,
#     "lap_start": 1,
#     "meeting_key": 1219,
#     "session_key": 9165,
#     "stint_number": 1,
#     "tyre_age_at_start": 3

def get_tyre_stints(session_keys):
    all_tyres = []
    
    for session_key in session_keys:
        url = f"https://api.openf1.org/v1/stints?session_key={session_key}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                for stint in data:
                    tyre_record = {
                        'session_key': session_key,
                        'driver_number': stint.get('driver_number'),
                        'stint_number': stint.get('stint_number'),
                        'compound': stint.get('compound'),
                        'lap_start': stint.get('lap_start'),
                        'lap_end': stint.get('lap_end'),
                        'tyre_age_at_start': stint.get('tyre_age_at_start'),
                        'meeting_key': stint.get('meeting_key')
                    }
                    all_tyres.append(tyre_record)
                
                print(f"Session {session_key}: Pobrano {len(data)} stints")
            else:
                print(f"Session {session_key}: Brak danych o oponach")
        else:
            print(f"Error fetching tyre stints for session {session_key}: {response.status_code}")
        
        sleep(0.5)
    
    return pd.DataFrame(all_tyres)

In [ ]:
stints_tyre = get_tyre_stints(sessions_key_unique)
stints_tyre.head()

Session 9462: Pobrano 220 stints
Session 9463: Pobrano 217 stints
Session 9464: Pobrano 216 stints
Session 9465: Pobrano 79 stints
Session 9466: Pobrano 79 stints
Session 9467: Pobrano 84 stints
Session 9468: Pobrano 92 stints
Session 9472: Pobrano 63 stints
Session 9473: Pobrano 74 stints
Session 9474: Pobrano 77 stints
Session 9475: Pobrano 64 stints
Session 9476: Pobrano 93 stints
Session 9480: Pobrano 39 stints
Session 9481: Pobrano 81 stints
Session 9482: Pobrano 76 stints
Session 9483: Pobrano 86 stints
Session 9484: Pobrano 86 stints
Session 9488: Pobrano 55 stints
Session 9489: Pobrano 79 stints
Session 9490: Pobrano 30 stints
Session 9491: Pobrano 76 stints
Session 9492: Pobrano 75 stints
Session 9496: Pobrano 74 stints
Session 9663: Pobrano 71 stints
Session 9668: Pobrano 53 stints
Session 9672: Pobrano 21 stints
Session 9664: Pobrano 94 stints
Session 9673: Pobrano 60 stints
Session 9497: Pobrano 82 stints
Session 9502: Pobrano 58 stints
Session 9506: Pobrano 57 stints
Sessi

,session_key,driver_number,stint_number,compound,lap_start,lap_end,tyre_age_at_start,meeting_key
0,9462,63,1,MEDIUM,1.0,6.0,0,1228
1,9462,14,1,MEDIUM,1.0,2.0,0,1228
2,9462,23,1,SOFT,1.0,2.0,0,1228
3,9462,77,1,MEDIUM,1.0,1.0,0,1228
4,9462,31,1,HARD,1.0,1.0,0,1228


In [ ]:
# https://api.openf1.org/v1/weather
    # "air_temperature": 27.8,
    # "date": "2023-05-07T18:42:25.233000+00:00",
    # "humidity": 58,
    # "meeting_key": 1208,
    # "pressure": 1018.7,
    # "rainfall": 0,
    # "session_key": 9078,
    # "track_temperature": 52.5,
    # "wind_direction": 136,
    # "wind_speed": 2.4

def get_weather_data(session_keys):
    all_weather = []
    
    for session_key in session_keys:
        url = f"https://api.openf1.org/v1/weather?session_key={session_key}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                for weather in data:
                    date_w, time_w = format_datetime_fields(weather['date'])
                    weather_record = {
                        'session_key': session_key,
                        'meeting_key': weather.get('meeting_key'),
                        'date': date_w,
                        'time': time_w,
                        'air_temperature': weather.get('air_temperature'),
                        'track_temperature': weather.get('track_temperature'),
                        'humidity': weather.get('humidity'),
                        'pressure': weather.get('pressure'),
                        'rainfall': weather.get('rainfall'),
                        'wind_direction': weather.get('wind_direction'),
                        'wind_speed': weather.get('wind_speed')
                    }
                    all_weather.append(weather_record)
                
                print(f"Session {session_key}: Pobrano {len(data)} rekordów pogody")
            else:
                print(f"Session {session_key}: Brak danych o pogodzie")
        else:
            print(f"Error fetching weather data for session {session_key}: {response.status_code}")
        
        sleep(0.5)
    
    return pd.DataFrame(all_weather)

In [ ]:
weather_session_df = get_weather_data(sessions_key_unique)
weather_session_df.head()

Session 9462: Pobrano 279 rekordów pogody
Session 9463: Pobrano 560 rekordów pogody
Session 9464: Pobrano 552 rekordów pogody
Session 9465: Pobrano 82 rekordów pogody
Session 9466: Pobrano 82 rekordów pogody
Session 9467: Pobrano 82 rekordów pogody
Session 9468: Pobrano 77 rekordów pogody
Session 9472: Pobrano 157 rekordów pogody
Session 9473: Pobrano 80 rekordów pogody
Session 9474: Pobrano 93 rekordów pogody
Session 9475: Pobrano 82 rekordów pogody
Session 9476: Pobrano 94 rekordów pogody
Session 9480: Pobrano 146 rekordów pogody
Session 9481: Pobrano 81 rekordów pogody
Session 9482: Pobrano 82 rekordów pogody
Session 9483: Pobrano 81 rekordów pogody
Session 9484: Pobrano 78 rekordów pogody
Session 9488: Pobrano 144 rekordów pogody
Session 9489: Pobrano 80 rekordów pogody
Session 9490: Pobrano 86 rekordów pogody
Session 9491: Pobrano 79 rekordów pogody
Session 9492: Pobrano 76 rekordów pogody
Session 9496: Pobrano 181 rekordów pogody
Session 9663: Pobrano 82 rekordów pogody
Session 9

,session_key,meeting_key,date,time,air_temperature,track_temperature,humidity,pressure,rainfall,wind_direction,wind_speed
0,9462,1228,2024-02-21,06:55:35.254000,21.8,28.6,61.0,1020.5,0,109,0.9
1,9462,1228,2024-02-21,06:56:35.271000,21.8,28.6,61.0,1020.5,0,1,1.1
2,9462,1228,2024-02-21,06:57:35.289000,21.9,28.9,60.0,1020.5,0,48,0.7
3,9462,1228,2024-02-21,06:58:35.293000,22.0,29.2,59.0,1020.4,0,9,1.1
4,9462,1228,2024-02-21,06:59:35.279000,22.1,29.4,58.0,1020.4,0,55,1.1


In [ ]:
# grupowanie danych pogodowych według sesji i obliczenie statystyk

def summarize_weather(group):
    return pd.Series({
        'air_temp_mean': group['air_temperature'].mean(),
        'air_temp_min': group['air_temperature'].min(),
        'air_temp_max': group['air_temperature'].max(),
        'track_temp_mean': group['track_temperature'].mean(),
        'humidity_mean': group['humidity'].mean(),
        'rainfall_total': group['rainfall'].sum(),
        'rainfall_max': group['rainfall'].max(),
        'wind_speed_mean': group['wind_speed'].mean(),
        'pressure_mean': group['pressure'].mean(),
        'weather_measurements': len(group)
    })


In [ ]:
gruouped_weather = weather_session_df.groupby('session_key')
gruouped_weather_summary = gruouped_weather.apply(summarize_weather).reset_index()

gruouped_weather_summary.head()

C:\Users\Sylwia\AppData\Local\Temp\ipykernel_5672\457088065.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gruouped_weather_summary = gruouped_weather.apply(summarize_weather).reset_index()


,session_key,air_temp_mean,air_temp_min,air_temp_max,track_temp_mean,humidity_mean,rainfall_total,rainfall_max,wind_speed_mean,pressure_mean,weather_measurements
0,9461,27.948780,27.8,28.3,41.568293,56.195122,0.0,0.0,2.112195,1016.978049,82.0
1,9462,23.606452,21.8,24.2,36.656272,39.200717,0.0,0.0,2.713620,1019.352688,279.0
2,9463,22.206429,20.0,23.3,32.737143,45.005357,0.0,0.0,2.730536,1019.099286,560.0
3,9464,23.544746,21.1,24.9,33.844928,49.336957,0.0,0.0,1.650543,1017.577717,552.0
4,9465,19.373171,19.0,19.8,34.215854,38.743902,0.0,0.0,3.378049,1016.101220,82.0


In [ ]:
# z sessions tylko sesje typu Race

all_race_df = all_sessions_df[all_sessions_df['session_type'] == 'Race']
all_race_df.head()



,meeting_key,year,session_key,session_type,session_name,date_start,time_start,date_end,time_end,gmt_offset,location,country_name,country_code,circuit_short_name
7,1229,2024,9472,Race,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir
12,1230,2024,9480,Race,Race,2024-03-09,17:00:00,2024-03-09,19:00:00,03:00:00,Jeddah,Saudi Arabia,KSA,Jeddah
17,1231,2024,9488,Race,Race,2024-03-24,04:00:00,2024-03-24,06:00:00,11:00:00,Melbourne,Australia,AUS,Melbourne
22,1232,2024,9496,Race,Race,2024-04-07,05:00:00,2024-04-07,07:00:00,09:00:00,Suzuka,Japan,JPN,Suzuka
25,1233,2024,9672,Race,Sprint,2024-04-20,03:00:00,2024-04-20,03:30:00,08:00:00,Shanghai,China,CHN,Shanghai


In [ ]:
# z sessions tylko sesje typu Qualifying
all_qualifying_df = all_sessions_df[all_sessions_df['session_name'] == 'Qualifying']
all_qualifying_df.head()

,meeting_key,year,session_key,session_type,session_name,date_start,time_start,date_end,time_end,gmt_offset,location,country_name,country_code,circuit_short_name
6,1229,2024,9468,Qualifying,Qualifying,2024-03-01,16:00:00,2024-03-01,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir
11,1230,2024,9476,Qualifying,Qualifying,2024-03-08,17:00:00,2024-03-08,18:00:00,03:00:00,Jeddah,Saudi Arabia,KSA,Jeddah
16,1231,2024,9484,Qualifying,Qualifying,2024-03-23,05:00:00,2024-03-23,06:00:00,11:00:00,Melbourne,Australia,AUS,Melbourne
21,1232,2024,9492,Qualifying,Qualifying,2024-04-06,06:00:00,2024-04-06,07:00:00,09:00:00,Suzuka,Japan,JPN,Suzuka
26,1233,2024,9664,Qualifying,Qualifying,2024-04-20,07:00:00,2024-04-20,08:00:00,08:00:00,Shanghai,China,CHN,Shanghai


In [ ]:
# polaczenie wyscigow i kierowcow

df_driver = drivers_df
df_gp = all_meetings_df
df_races = all_race_df

races_session_keys = df_races[['session_key']].drop_duplicates()
df_driver_race = df_driver.merge(races_session_keys, on='session_key', how='inner')

print(df_driver_race['session_key'].unique())
print(df_races['session_key'].unique())

# polaczenie driver z race
df_merged = df_driver_race.merge(df_races, on=['meeting_key', 'session_key'], how='left')
df_merged.head()

[9472 9480 9488 9496 9672 9673 9506 9507 9515 9523 9531 9539 9549 9550
 9558 9566 9574 9582 9590 9598 9606 9616 9617 9625 9635 9636 9644 9654
 9655 9662]
[9472 9480 9488 9496 9672 9673 9506 9507 9515 9523 9531 9539 9549 9550
 9558 9566 9574 9582 9590 9598 9606 9616 9617 9625 9635 9636 9644 9654
 9655 9662]


,driver_number,broadcast_name,full_name,first_name,last_name,name_acronym,country_code_x,team_name,meeting_key,session_key,...,session_name,date_start,time_start,date_end,time_end,gmt_offset,location,country_name,country_code_y,circuit_short_name
0,1,M VERSTAPPEN,Max VERSTAPPEN,Max,Verstappen,VER,NED,Red Bull Racing,1229,9472,...,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir
1,2,L SARGEANT,Logan SARGEANT,Logan,Sargeant,SAR,USA,Williams,1229,9472,...,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir
2,3,D RICCIARDO,Daniel RICCIARDO,Daniel,Ricciardo,RIC,AUS,RB,1229,9472,...,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir
3,4,L NORRIS,Lando NORRIS,Lando,Norris,NOR,GBR,McLaren,1229,9472,...,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir
4,10,P GASLY,Pierre GASLY,Pierre,Gasly,GAS,FRA,Alpine,1229,9472,...,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir


In [ ]:
results_qualifying = get_final_session_results(all_qualifying_df['session_key'].unique(), include_driver_details=True)
results_qualifying.head()

Session 9468: Pobrano 20 końcowych pozycji
Session 9476: Pobrano 20 końcowych pozycji
Session 9484: Pobrano 19 końcowych pozycji
Session 9492: Pobrano 20 końcowych pozycji
Session 9664: Pobrano 20 końcowych pozycji
Session 9498: Pobrano 20 końcowych pozycji
Session 9511: Pobrano 20 końcowych pozycji
Session 9519: Pobrano 20 końcowych pozycji
Session 9527: Pobrano 20 końcowych pozycji
Session 9535: Pobrano 20 końcowych pozycji
Session 9541: Pobrano 20 końcowych pozycji
Session 9554: Pobrano 20 końcowych pozycji
Session 9562: Pobrano 20 końcowych pozycji
Session 9570: Pobrano 20 końcowych pozycji
Session 9578: Pobrano 20 końcowych pozycji
Session 9586: Pobrano 20 końcowych pozycji
Session 9594: Pobrano 20 końcowych pozycji
Session 9602: Pobrano 20 końcowych pozycji
Session 9608: Pobrano 20 końcowych pozycji
Session 9621: Pobrano 20 końcowych pozycji
Session 9627: Pobrano 20 końcowych pozycji
Session 9640: Pobrano 20 końcowych pozycji
Session 9646: Pobrano 20 końcowych pozycji
Session 965

,session_key,driver_number,position,date,time,meeting_key,session_name,year,location,country_name,circuit_short_name,broadcast_name,full_name,team_name
0,9468,55,4,2024-03-01,17:01:09.329000,1229,Qualifying,2024,Sakhir,Bahrain,Sakhir,C SAINZ,Carlos SAINZ,Ferrari
1,9468,14,6,2024-03-01,17:01:09.329000,1229,Qualifying,2024,Sakhir,Bahrain,Sakhir,F ALONSO,Fernando ALONSO,Aston Martin
2,9468,1,1,2024-03-01,17:01:09.329000,1229,Qualifying,2024,Sakhir,Bahrain,Sakhir,M VERSTAPPEN,Max VERSTAPPEN,Red Bull Racing
3,9468,16,2,2024-03-01,17:01:09.329000,1229,Qualifying,2024,Sakhir,Bahrain,Sakhir,C LECLERC,Charles LECLERC,Ferrari
4,9468,4,7,2024-03-01,17:01:09.329000,1229,Qualifying,2024,Sakhir,Bahrain,Sakhir,L NORRIS,Lando NORRIS,McLaren


In [ ]:
# dodanie do df_merged wyników kwalifikacji z zachowaniem wszytskich kolumn df_merged a dodanie tylko quali_position
df_final = []

df_final = df_merged.merge(
    results_qualifying[['meeting_key', 'driver_number', 'position']],
    on=['meeting_key', 'driver_number'],
    how='left'
).rename(columns={'position': 'qualifying_position'})

df_final.head()

,driver_number,broadcast_name,full_name,first_name,last_name,name_acronym,country_code_x,team_name,meeting_key,session_key,...,date_start,time_start,date_end,time_end,gmt_offset,location,country_name,country_code_y,circuit_short_name,qualifying_position
0,1,M VERSTAPPEN,Max VERSTAPPEN,Max,Verstappen,VER,NED,Red Bull Racing,1229,9472,...,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir,1
1,2,L SARGEANT,Logan SARGEANT,Logan,Sargeant,SAR,USA,Williams,1229,9472,...,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir,18
2,3,D RICCIARDO,Daniel RICCIARDO,Daniel,Ricciardo,RIC,AUS,RB,1229,9472,...,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir,14
3,4,L NORRIS,Lando NORRIS,Lando,Norris,NOR,GBR,McLaren,1229,9472,...,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir,7
4,10,P GASLY,Pierre GASLY,Pierre,Gasly,GAS,FRA,Alpine,1229,9472,...,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir,20


In [ ]:
df_race_results = results_df[results_df['session_name'] == 'Race']
df_race_results.head()

,session_key,driver_number,position,date,time,meeting_key,session_name,year,location,country_name,circuit_short_name,broadcast_name,full_name,team_name
140,9472,1,1,2024-03-02,16:28:00.504000,1229,Race,2024,Sakhir,Bahrain,Sakhir,M VERSTAPPEN,Max VERSTAPPEN,Red Bull Racing
141,9472,16,4,2024-03-02,16:28:00.504000,1229,Race,2024,Sakhir,Bahrain,Sakhir,C LECLERC,Charles LECLERC,Ferrari
142,9472,63,5,2024-03-02,16:28:00.504000,1229,Race,2024,Sakhir,Bahrain,Sakhir,G RUSSELL,George RUSSELL,Mercedes
143,9472,55,3,2024-03-02,16:28:00.504000,1229,Race,2024,Sakhir,Bahrain,Sakhir,C SAINZ,Carlos SAINZ,Ferrari
144,9472,11,2,2024-03-02,16:28:00.504000,1229,Race,2024,Sakhir,Bahrain,Sakhir,S PEREZ,Sergio PEREZ,Red Bull Racing


In [ ]:
# doodanie do df_final pozycji startowych z zachowaniem wszytskich kolumn df_final a dod
df_starting_positions = starting_position_df[starting_position_df['session_name'] == 'Race']
df_starting_positions.head()

,session_key,driver_number,position,date,time,meeting_key,session_name,year,location,country_name,broadcast_name,full_name,team_name
140,9472,1,1,2024-03-02,14:03:47.739000,1229,Race,2024,Sakhir,Bahrain,M VERSTAPPEN,Max VERSTAPPEN,Red Bull Racing
141,9472,16,2,2024-03-02,14:03:47.739000,1229,Race,2024,Sakhir,Bahrain,C LECLERC,Charles LECLERC,Ferrari
142,9472,63,3,2024-03-02,14:03:47.739000,1229,Race,2024,Sakhir,Bahrain,G RUSSELL,George RUSSELL,Mercedes
143,9472,55,4,2024-03-02,14:03:47.739000,1229,Race,2024,Sakhir,Bahrain,C SAINZ,Carlos SAINZ,Ferrari
144,9472,11,5,2024-03-02,14:03:47.739000,1229,Race,2024,Sakhir,Bahrain,S PEREZ,Sergio PEREZ,Red Bull Racing


In [ ]:
# Dodanie pozycji startowej z wyścigu i kwalifikacji do df_final
race_starting = starting_position_df[starting_position_df['session_name'] == 'Race'][['meeting_key', 'driver_number', 'position']].rename(columns={'position': 'starting_position'})
qualifying_starting = starting_position_df[starting_position_df['session_name'] == 'Qualifying'][['meeting_key', 'driver_number', 'position']].rename(columns={'position': 'qualifying_position'})
df_final = df_final.merge(race_starting, on=['meeting_key', 'driver_number'], how='left')
df_final = df_final.merge(qualifying_starting, on=['meeting_key', 'driver_number'], how='left')
df_final.head()

,driver_number,broadcast_name,full_name,first_name,last_name,name_acronym,country_code_x,team_name,meeting_key,session_key,...,date_end,time_end,gmt_offset,location,country_name,country_code_y,circuit_short_name,qualifying_position_x,starting_position,qualifying_position_y
0,1,M VERSTAPPEN,Max VERSTAPPEN,Max,Verstappen,VER,NED,Red Bull Racing,1229,9472,...,2024-03-02,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir,1,1,3
1,2,L SARGEANT,Logan SARGEANT,Logan,Sargeant,SAR,USA,Williams,1229,9472,...,2024-03-02,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir,18,18,19
2,3,D RICCIARDO,Daniel RICCIARDO,Daniel,Ricciardo,RIC,AUS,RB,1229,9472,...,2024-03-02,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir,14,14,11
3,4,L NORRIS,Lando NORRIS,Lando,Norris,NOR,GBR,McLaren,1229,9472,...,2024-03-02,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir,7,7,5
4,10,P GASLY,Pierre GASLY,Pierre,Gasly,GAS,FRA,Alpine,1229,9472,...,2024-03-02,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir,20,20,20


In [ ]:
def remove_duplicate_columns(df, column_name):
    cols = [col for col in df.columns if col == column_name]
    if len(cols) > 1:
        first = df.columns.get_loc(column_name)
        df = df.loc[:, ~df.columns.duplicated()]
    return df

In [ ]:
df_final = df_final.merge(
    df_race_results[['meeting_key', 'driver_number', 'position']],
    on=['meeting_key', 'driver_number'],
    how='left'
).rename(columns={'position': 'final_position'})

df_final = remove_duplicate_columns(df_final, 'final_position')
df_final.head()

,driver_number,broadcast_name,full_name,first_name,last_name,name_acronym,country_code_x,team_name,meeting_key,session_key,...,time_end,gmt_offset,location,country_name,country_code_y,circuit_short_name,qualifying_position_x,starting_position,qualifying_position_y,final_position
0,1,M VERSTAPPEN,Max VERSTAPPEN,Max,Verstappen,VER,NED,Red Bull Racing,1229,9472,...,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir,1,1,3,1
1,2,L SARGEANT,Logan SARGEANT,Logan,Sargeant,SAR,USA,Williams,1229,9472,...,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir,18,18,19,20
2,3,D RICCIARDO,Daniel RICCIARDO,Daniel,Ricciardo,RIC,AUS,RB,1229,9472,...,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir,14,14,11,13
3,4,L NORRIS,Lando NORRIS,Lando,Norris,NOR,GBR,McLaren,1229,9472,...,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir,7,7,5,6
4,10,P GASLY,Pierre GASLY,Pierre,Gasly,GAS,FRA,Alpine,1229,9472,...,17:00:00,03:00:00,Sakhir,Bahrain,BRN,Sakhir,20,20,20,18


In [ ]:
# policz zwyciestwa kazdego kierowcy do odbywajacego sie meeting, 
# tak aby policz tylko zwyciestwa przed rozpatrywanym meeting
# jako parametr funkcja przyjmuje tylko ramke ze wszytskimi wynikami sesji

def count_wins_before_meeting(race_results):
    # Posortuj dane rosnąco wg meeting_key (lub daty, jeśli dostępna)
    race_sorted = race_results.sort_values('meeting_key').reset_index(drop=True).copy()
    
    # Słownik do zliczania zwycięstw kierowców
    wins_count = {}
    
    # Lista na wyniki dla kolumny wins_before
    wins_before_list = []
    
    # Iteruj po posortowanych danych
    for idx, row in race_sorted.iterrows():
        driver = row['driver_number']
        meeting = row['meeting_key']
        
        # Pobierz aktualną liczbę zwycięstw przed tym meetingiem
        wins_before = wins_count.get(driver, 0)
        wins_before_list.append(wins_before)
        
        # Jeśli kierowca wygrał (position == 1), zwiększ licznik zwycięstw
        if row['final_position'] == 1:
            wins_count[driver] = wins_before + 1
    
    # Dodaj nową kolumnę do kopii ramki danych
    race_sorted['wins_before'] = wins_before_list
    
    return race_sorted


test = count_wins_before_meeting(df_final)

#wyswitl tylko dla driver_number 1
print(test[test['driver_number'] == 1][['meeting_key', 'driver_number', 'final_position', 'wins_before']])

     meeting_key  driver_number  final_position  wins_before
0           1229              1               1            0
35          1230              1               1            1
57          1231              1              19            2
74          1232              1               1            2
97          1233              1               1            3
109         1233              1               1            4
137         1234              1               2            5
140         1234              1               2            5
177         1235              1               1            5
194         1236              1               6            6
217         1237              1               1            6
231         1238              1               1            7
257         1239              1               5            8
261         1239              1               5            8
297         1240              1               2            8
309         1241        

In [ ]:
# merge z df_final
df_final = df_final.merge(
    test[['meeting_key', 'driver_number', 'wins_before']],
    on=['meeting_key', 'driver_number'],
    how='left'
)
df_final.head()


,driver_number,broadcast_name,full_name,first_name,last_name,name_acronym,country_code_x,team_name,meeting_key,session_key,...,gmt_offset,location,country_name,country_code_y,circuit_short_name,qualifying_position_x,starting_position,qualifying_position_y,final_position,wins_before
0,1,M VERSTAPPEN,Max VERSTAPPEN,Max,Verstappen,VER,NED,Red Bull Racing,1229,9472,...,03:00:00,Sakhir,Bahrain,BRN,Sakhir,1,1,3,1,0
1,2,L SARGEANT,Logan SARGEANT,Logan,Sargeant,SAR,USA,Williams,1229,9472,...,03:00:00,Sakhir,Bahrain,BRN,Sakhir,18,18,19,20,0
2,3,D RICCIARDO,Daniel RICCIARDO,Daniel,Ricciardo,RIC,AUS,RB,1229,9472,...,03:00:00,Sakhir,Bahrain,BRN,Sakhir,14,14,11,13,0
3,4,L NORRIS,Lando NORRIS,Lando,Norris,NOR,GBR,McLaren,1229,9472,...,03:00:00,Sakhir,Bahrain,BRN,Sakhir,7,7,5,6,0
4,10,P GASLY,Pierre GASLY,Pierre,Gasly,GAS,FRA,Alpine,1229,9472,...,03:00:00,Sakhir,Bahrain,BRN,Sakhir,20,20,20,18,0


In [ ]:
# oblicz liczbe pit stopów dla każdego kierowcy w danym wyścigu i sredni czas trwania pit stopu

def calculate_pit_stops_stats(pit_stops_df):
    # Grupuj po meeting_key i driver_number, zlicz liczbę pit stopów i oblicz średni czas trwania
    pit_stats = (
        pit_stops_df
        .groupby(['session_key', 'driver_number'])
        .agg(
            pit_stop_count=('pit_duration', 'count'),
            avg_pit_duration=('pit_duration', 'mean')
        )
        .reset_index()
    )
    
    return pit_stats

In [ ]:
race_sessions_key_unique = df_final['session_key'].unique()
race_sessions_key_unique


array([9472, 9480, 9488, 9496, 9672, 9673, 9506, 9507, 9515, 9523, 9531,
       9539, 9549, 9550, 9558, 9566, 9574, 9582, 9590, 9598, 9606, 9616,
       9617, 9625, 9635, 9636, 9644, 9654, 9655, 9662])

In [ ]:
df_race_pit_stops = pit_stops_df[pit_stops_df['session_key'].isin(race_sessions_key_unique)]


In [ ]:
df_race_pit_stops_stats = calculate_pit_stops_stats(df_race_pit_stops)
df_race_pit_stops_stats.head()

,session_key,driver_number,pit_stop_count,avg_pit_duration
0,9472,1,2,24.600000
1,9472,2,3,28.666667
2,9472,3,2,24.850000
3,9472,4,2,24.450000
4,9472,10,3,27.166667


In [ ]:
#merge pit stop stats z df_final
df_final = df_final.merge(
    df_race_pit_stops_stats,
    on=['session_key', 'driver_number'],
    how='left'
).rename(columns={
    'pit_stop_count': 'pit_stops_count',
    'avg_pit_duration': 'avg_pit_stop_duration'
})

#sortowanie df_final po meeting_key i driver_number


In [ ]:
df_final = df_final.sort_values(by=['session_key', 'driver_number']).reset_index(drop=True)
df_final.head()

,driver_number,broadcast_name,full_name,first_name,last_name,name_acronym,country_code_x,team_name,meeting_key,session_key,...,country_name,country_code_y,circuit_short_name,qualifying_position_x,starting_position,qualifying_position_y,final_position,wins_before,pit_stops_count,avg_pit_stop_duration
0,1,M VERSTAPPEN,Max VERSTAPPEN,Max,Verstappen,VER,NED,Red Bull Racing,1229,9472,...,Bahrain,BRN,Sakhir,1,1,3,1,0,2.0,24.600000
1,2,L SARGEANT,Logan SARGEANT,Logan,Sargeant,SAR,USA,Williams,1229,9472,...,Bahrain,BRN,Sakhir,18,18,19,20,0,3.0,28.666667
2,3,D RICCIARDO,Daniel RICCIARDO,Daniel,Ricciardo,RIC,AUS,RB,1229,9472,...,Bahrain,BRN,Sakhir,14,14,11,13,0,2.0,24.850000
3,4,L NORRIS,Lando NORRIS,Lando,Norris,NOR,GBR,McLaren,1229,9472,...,Bahrain,BRN,Sakhir,7,7,5,6,0,2.0,24.450000
4,10,P GASLY,Pierre GASLY,Pierre,Gasly,GAS,FRA,Alpine,1229,9472,...,Bahrain,BRN,Sakhir,20,20,20,18,0,3.0,27.166667


In [ ]:
# Ensure there are no duplicate columns before printing
df_final_clean = df_final.loc[:, ~df_final.columns.duplicated()]
print(df_final_clean[df_final_clean['pit_stops_count'] > 0][['meeting_key', 'session_key', 'driver_number', 'pit_stops_count', 'avg_pit_stop_duration']].head(10))

   meeting_key  session_key  driver_number  pit_stops_count  \
0         1229         9472              1              2.0   
1         1229         9472              2              3.0   
2         1229         9472              3              2.0   
3         1229         9472              4              2.0   
4         1229         9472             10              3.0   
5         1229         9472             11              2.0   
6         1229         9472             14              2.0   
7         1229         9472             16              2.0   
8         1229         9472             18              2.0   
9         1229         9472             20              2.0   

   avg_pit_stop_duration  
0              24.600000  
1              28.666667  
2              24.850000  
3              24.450000  
4              27.166667  
5              24.500000  
6              24.900000  
7              24.050000  
8              24.450000  
9              25.150000  


In [ ]:
df_final.head()
df_copy = df_final.copy()

# zastap wszystkie wartosci NaN w kolumnach pit_stops_count i avg_pit_stop_duration zerami
df_copy['pit_stops_count'] = df_copy['pit_stops_count'].fillna(0).astype(int)
df_copy['avg_pit_stop_duration'] = df_copy['avg_pit_stop_duration'].fillna(0).astype(float)



In [ ]:
print(df_copy.head())

   driver_number broadcast_name         full_name first_name   last_name  \
0              1   M VERSTAPPEN    Max VERSTAPPEN        Max  Verstappen   
1              2     L SARGEANT    Logan SARGEANT      Logan    Sargeant   
2              3    D RICCIARDO  Daniel RICCIARDO     Daniel   Ricciardo   
3              4       L NORRIS      Lando NORRIS      Lando      Norris   
4             10        P GASLY      Pierre GASLY     Pierre       Gasly   

  name_acronym country_code_x        team_name  meeting_key  session_key  ...  \
0          VER            NED  Red Bull Racing         1229         9472  ...   
1          SAR            USA         Williams         1229         9472  ...   
2          RIC            AUS               RB         1229         9472  ...   
3          NOR            GBR          McLaren         1229         9472  ...   
4          GAS            FRA           Alpine         1229         9472  ...   

   country_code_y circuit_short_name qualifying_position

In [ ]:
starting_position_df.head()

,session_key,driver_number,position,date,time,meeting_key,session_name,year,location,country_name,broadcast_name,full_name,team_name
0,9462,1,1,2024-02-21,06:55:17.159000,1228,Practice 1,2024,Sakhir,Bahrain,M VERSTAPPEN,Max VERSTAPPEN,Red Bull Racing
1,9462,2,2,2024-02-21,06:55:17.159000,1228,Practice 1,2024,Sakhir,Bahrain,L SARGEANT,Logan SARGEANT,Williams
2,9462,3,3,2024-02-21,06:55:17.159000,1228,Practice 1,2024,Sakhir,Bahrain,D RICCIARDO,Daniel RICCIARDO,RB
3,9462,4,4,2024-02-21,06:55:17.159000,1228,Practice 1,2024,Sakhir,Bahrain,L NORRIS,Lando NORRIS,McLaren
4,9462,10,5,2024-02-21,06:55:17.159000,1228,Practice 1,2024,Sakhir,Bahrain,P GASLY,Pierre GASLY,Alpine


In [ ]:
# save datasets to csv files
df_copy.to_csv('f1_race_data_all.csv', index=False)
df_sessions = all_sessions_df

df_copy.head()

,driver_number,broadcast_name,full_name,first_name,last_name,name_acronym,country_code_x,team_name,meeting_key,session_key,...,country_code_y,circuit_short_name,qualifying_position_x,starting_position,qualifying_position_y,final_position,wins_before,pit_stops_count,avg_pit_stop_duration,position_category
0,1,M VERSTAPPEN,Max VERSTAPPEN,Max,Verstappen,VER,NED,Red Bull Racing,1229,9472,...,BRN,Sakhir,1,1,3,1,0,2,24.600000,winner
1,2,L SARGEANT,Logan SARGEANT,Logan,Sargeant,SAR,USA,Williams,1229,9472,...,BRN,Sakhir,18,18,19,20,0,3,28.666667,no_points
2,3,D RICCIARDO,Daniel RICCIARDO,Daniel,Ricciardo,RIC,AUS,RB,1229,9472,...,BRN,Sakhir,14,14,11,13,0,2,24.850000,no_points
3,4,L NORRIS,Lando NORRIS,Lando,Norris,NOR,GBR,McLaren,1229,9472,...,BRN,Sakhir,7,7,5,6,0,2,24.450000,points
4,10,P GASLY,Pierre GASLY,Pierre,Gasly,GAS,FRA,Alpine,1229,9472,...,BRN,Sakhir,20,20,20,18,0,3,27.166667,no_points


In [ ]:
def categorize_position(pos):
    if pos == 1:
        return 'winner'
    elif pos in [2, 3]:
        return 'top3'
    elif 4 <= pos <= 10:
        return 'points'
    else:
        return 'no_points'

In [ ]:
# Tworzymy nową kolumnę target
df_final['position_category'] = df_final['final_position'].apply(categorize_position)

In [ ]:
df_final.head()
back = df_copy.copy()

In [ ]:
df_copy['position_category'] = df_copy['final_position'].apply(categorize_position)

In [ ]:
features_to_drop = [ 'full_name', 'first_name', 'last_name',
                    'name_acronym', 'team_name', 'meeting_key', 'session_key', 'location', 'country_name',
                    'country_code_y', 'circuit_short_name', 'final_position', 'position_category','session_type']

#mozna usunac session tyype... bo w modelku niczego to nie zmiana bo wszytskie taka maja 
#ale bedzie potrzeba ogolem 


In [ ]:
X = df_copy.drop(columns=features_to_drop)
y = df_copy['position_category']

data = X.copy()
data['position_category'] = y

data.head()

,driver_number,broadcast_name,country_code_x,year,session_name,date_start,time_start,date_end,time_end,gmt_offset,qualifying_position_x,starting_position_x,qualifying_position_y,wins_before,pit_stops_count,avg_pit_stop_duration,starting_position_y,starting_position,position_category
0,1,M VERSTAPPEN,NED,2024,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,1,1,3,0,2,24.600000,1,1,winner
1,2,L SARGEANT,USA,2024,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,18,18,19,0,3,28.666667,18,18,no_points
2,3,D RICCIARDO,AUS,2024,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,14,14,11,0,2,24.850000,14,14,no_points
3,4,L NORRIS,GBR,2024,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,7,7,5,0,2,24.450000,7,7,points
4,10,P GASLY,FRA,2024,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,20,20,20,0,3,27.166667,20,20,no_points


In [ ]:
#usuwa quali_x i y 
data['qualifying_position'] = data['qualifying_position_y']
data = data.drop(columns=['qualifying_position_x', 'qualifying_position_y'])
data.head()

,driver_number,broadcast_name,country_code_x,year,session_name,date_start,time_start,date_end,time_end,gmt_offset,starting_position_x,wins_before,pit_stops_count,avg_pit_stop_duration,starting_position_y,starting_position,position_category,qualifying_position
0,1,M VERSTAPPEN,NED,2024,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,1,0,2,24.600000,1,1,winner,3
1,2,L SARGEANT,USA,2024,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,18,0,3,28.666667,18,18,no_points,19
2,3,D RICCIARDO,AUS,2024,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,14,0,2,24.850000,14,14,no_points,11
3,4,L NORRIS,GBR,2024,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,7,0,2,24.450000,7,7,points,5
4,10,P GASLY,FRA,2024,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,20,0,3,27.166667,20,20,no_points,20


In [ ]:
#zamien kolejnosc zeby position_category byla ostatnia kolumna
cols = data.columns.tolist()
cols = [col for col in cols if col != 'position_category'] + ['position_category']
data = data[cols]
data.head()

,driver_number,broadcast_name,country_code_x,year,session_name,date_start,time_start,date_end,time_end,gmt_offset,starting_position_x,wins_before,pit_stops_count,avg_pit_stop_duration,starting_position_y,starting_position,qualifying_position,position_category
0,1,M VERSTAPPEN,NED,2024,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,1,0,2,24.600000,1,1,3,winner
1,2,L SARGEANT,USA,2024,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,18,0,3,28.666667,18,18,19,no_points
2,3,D RICCIARDO,AUS,2024,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,14,0,2,24.850000,14,14,11,no_points
3,4,L NORRIS,GBR,2024,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,7,0,2,24.450000,7,7,5,points
4,10,P GASLY,FRA,2024,Race,2024-03-02,15:00:00,2024-03-02,17:00:00,03:00:00,20,0,3,27.166667,20,20,20,no_points


In [ ]:
# do csv zapisz x oraz y do pliku f1_data.csv
data.to_csv('f1_data.csv', index=False)

In [ ]:
# wyswitel informacje o zbiorze danych
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

#wyswietl cechy
print("Features in X:", X.columns.tolist())
# wyswietl unikalne kategorie pozycji
print("Unique position categories in y:", y.unique())


Shape of X: (839, 18)
Shape of y: (839,)
Features in X: ['driver_number', 'broadcast_name', 'country_code_x', 'year', 'session_name', 'date_start', 'time_start', 'date_end', 'time_end', 'gmt_offset', 'qualifying_position_x', 'starting_position_x', 'qualifying_position_y', 'wins_before', 'pit_stops_count', 'avg_pit_stop_duration', 'starting_position_y', 'starting_position']
Unique position categories in y: ['winner' 'no_points' 'points' 'top3']


In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

for col in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, log_loss, mean_squared_error


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y, test_size=0.3, stratify=y, random_state=42)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Zakładam, że y_train i y_valid są dostępne globalnie

In [ ]:
def compute_specificity(conf_mat):
    specificity = []
    for i in range(conf_mat.shape[0]):
        TP = conf_mat[i, i]
        FP = conf_mat[:, i].sum() - TP
        FN = conf_mat[i, :].sum() - TP
        TN = conf_mat.sum() - (TP + FP + FN)
        spec = TN / (TN + FP) if (TN + FP) > 0 else 0
        specificity.append(spec)
    return specificity

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

def objective(trial):
    # === Hiperparametry ===
    rf_n_estimators = trial.suggest_int("rf_n_estimators", 50, 150)
    rf_max_depth = trial.suggest_int("rf_max_depth", 3, 15)

    gb_n_estimators = trial.suggest_int("gb_n_estimators", 50, 150)
    gb_learning_rate = trial.suggest_float("gb_learning_rate", 0.01, 0.3)
    gb_max_depth = trial.suggest_int("gb_max_depth", 2, 10)

    svm_C = trial.suggest_float("svm_C", 0.1, 10.0, log=True)
    svm_kernel = trial.suggest_categorical("svm_kernel", ["linear", "rbf"])

    # === Modele ===
    clf_rf = RandomForestClassifier(n_estimators=rf_n_estimators, max_depth=rf_max_depth, random_state=42)
    clf_gb = GradientBoostingClassifier(n_estimators=gb_n_estimators, learning_rate=gb_learning_rate,
                                        max_depth=gb_max_depth, random_state=42)
    clf_svm = SVC(C=svm_C, kernel=svm_kernel, probability=True, random_state=42)

    # === Ensemble ===
    ensemble = VotingClassifier(
        estimators=[('rf', clf_rf), ('gb', clf_gb), ('svm', clf_svm)],
        voting='soft'
    )

    # === Label encoding ===
    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train)
    y_valid_enc = le.transform(y_valid)

    class_labels = le.classes_
    print("Klasy w macierzy pomyłek:", class_labels)

    # === k-Fold Cross-Validation ===
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    acc_list, prec_list, rec_list, f1_list, spec_list = [], [], [], [], []
    mse_list, rmse_list, logloss_list = [], [], []

    # globalna macierz pomyłek
    conf_mat_sum = np.zeros((len(np.unique(y_train_enc)), len(np.unique(y_train_enc))), dtype=int)

    for train_index, test_index in kf.split(X_train, y_train_enc):
        X_tr, X_te = X_train[train_index], X_train[test_index]
        y_tr, y_te = y_train_enc[train_index], y_train_enc[test_index]

        ensemble.fit(X_tr, y_tr)
        y_pred_cv = ensemble.predict(X_te)
        y_proba_cv = ensemble.predict_proba(X_te)

        conf_mat_cv = confusion_matrix(y_te, y_pred_cv)
        conf_mat_sum += conf_mat_cv  # sumowanie po wszystkich foldach

        acc_list.append(accuracy_score(y_te, y_pred_cv))
        prec_list.append(precision_score(y_te, y_pred_cv, average="micro", zero_division=0))
        rec_list.append(recall_score(y_te, y_pred_cv, average="micro", zero_division=0))
        f1_list.append(f1_score(y_te, y_pred_cv, average="micro", zero_division=0))

        spec_cv = np.mean(compute_specificity(conf_mat_cv))
        spec_list.append(spec_cv)

        mse_list.append(mean_squared_error(y_te, y_pred_cv))
        rmse_list.append(np.sqrt(mse_list[-1]))
        logloss_list.append(log_loss(y_te, y_proba_cv))

    # === Średnie i SD dla metryk ===
    acc, acc_std = np.mean(acc_list), np.std(acc_list, ddof=0)
    prec, prec_std = np.mean(prec_list), np.std(prec_list, ddof=0)
    rec, rec_std = np.mean(rec_list), np.std(rec_list, ddof=0)
    f1, f1_std = np.mean(f1_list), np.std(f1_list, ddof=0)
    spec, spec_std = np.mean(spec_list), np.std(spec_list, ddof=0)

    mse, mse_std = np.mean(mse_list), np.std(mse_list, ddof=0)
    rmse, rmse_std = np.mean(rmse_list), np.std(rmse_list, ddof=0)
    logloss, logloss_std = np.mean(logloss_list), np.std(logloss_list, ddof=0)

    # === Zapis do triala ===
    trial.set_user_attr("accuracy", acc)
    trial.set_user_attr("accuracy_std", acc_std)
    trial.set_user_attr("precision", prec)
    trial.set_user_attr("precision_std", prec_std)
    trial.set_user_attr("recall", rec)
    trial.set_user_attr("recall_std", rec_std)
    trial.set_user_attr("f1_score", f1)
    trial.set_user_attr("f1_std", f1_std)
    trial.set_user_attr("specificity_avg", spec)
    trial.set_user_attr("specificity_std", spec_std)

    trial.set_user_attr("mse", mse)
    trial.set_user_attr("mse_std", mse_std)
    trial.set_user_attr("rmse", rmse)
    trial.set_user_attr("rmse_std", rmse_std)
    trial.set_user_attr("logloss", logloss)
    trial.set_user_attr("logloss_std", logloss_std)

    # zapis całej macierzy pomyłek (zsumowanej z 5 foldów)
    trial.set_user_attr("confusion_matrix", conf_mat_sum.tolist())

    return f1  # lub acc jeśli optymalizujesz dokładność


In [ ]:
# import optuna

# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=50)

In [ ]:
# best_params = study.best_trial.params

# print("🎯 Najlepsze hiperparametry:")
# for param, value in best_params.items():
#     print(f"{param}: {value}")


In [ ]:
# best_trial = study.best_trial

# print("\n📊 Metryki najlepszej próby:")
# print(f"F1-score (micro): {best_trial.user_attrs['f1_score']:.4f} ± {best_trial.user_attrs['f1_std']:.4f}")
# print(f"Accuracy: {best_trial.user_attrs['accuracy']:.4f} ± {best_trial.user_attrs['accuracy_std']:.4f}")
# print(f"Precision (micro): {best_trial.user_attrs['precision']:.4f} ± {best_trial.user_attrs['precision_std']:.4f}")
# print(f"Recall / Sensitivity (micro): {best_trial.user_attrs['recall']:.4f} ± {best_trial.user_attrs['recall_std']:.4f}")
# print(f"Specificity (avg): {best_trial.user_attrs['specificity_avg']:.4f} ± {best_trial.user_attrs['specificity_std']:.4f}")

# print(f"MSE: {best_trial.user_attrs['mse']:.4f} ± {best_trial.user_attrs['mse_std']:.4f}")
# print(f"RMSE: {best_trial.user_attrs['rmse']:.4f} ± {best_trial.user_attrs['rmse_std']:.4f}")
# print(f"LogLoss: {best_trial.user_attrs['logloss']:.4f} ± {best_trial.user_attrs['logloss_std']:.4f}")

# print("\n🧮 Confusion Matrix:")
# print(np.array(best_trial.user_attrs["confusion_matrix"]))


In [ ]:
# # Wyświetlenie etykiet klas w kolejności odpowiadającej macierzy pomyłek
# class_labels = le.classes_
# print("Klasy w macierzy pomyłek:", class_labels)

# # Macierz pomyłek zsumowana po 5 foldach
# conf_mat_global = np.array(best_trial.user_attrs["confusion_matrix"])
# print("Globalna macierz pomyłek:\n", conf_mat_global)

In [ ]:
!pip freeze > requirements.txt